In [1]:
# import Python packages
import pandas as pd
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
import scipy
import scipy.stats as ss
from skbio.stats.distance import permanova
import biom
from biom import load_table
from gemelli.rpca import rpca

# function that renders the figure in a notebook (instead of displaying a dump of the figure object)
%matplotlib inline

### See RPCA standalone python tutorial: 
# https://github.com/biocore/gemelli/blob/master/ipynb/tutorials/RPCA-moving-pictures-standalone-cli-and-api.ipynb

# Data loading

In [2]:
# read in biom table
biom_tbl = biom.load_table("/Users/annanguyen/16S_AD_South-Africa/Data/Tables/Relative_Abundance_Tables/df_16S_filtered_feature_table_rare_Genus_relative_abundance.biom")


In [3]:
# Load the metadata
metadata_path = '../Data/Metadata/updated_clean_ant_skin_metadata.tab'
metadata = pd.read_csv(metadata_path, sep='\t')

# removal of underscore in Sample IDs in metadata so it matches the formating of the biom table
metadata['#sample-id'] = metadata['#sample-id'].str.replace('_', '')


# Set Sample-ID as the index for the metadata dataframe 
metadata = metadata.set_index('#sample-id')


# Set the option to display all columns
pd.set_option('display.max_columns', None)

# Sorts the metadata df by sample ID and displays it
metadata_sorted = metadata.sort_values(by='#sample-id')
metadata_sorted.head()





,PlateNumber,PlateLocation,i5,i5Sequence,i7,i7Sequence,identifier,Sequence,Plate ID,Well location,Volume (ul),Lysozyme pretreatment,DNA extraction method,Purification method,Date of DNA extraction,pid,case_type,participant,area,sample_type,specimen,age_months,sex,enrolment_date,enrolment_season,hiv_exposure,hiv_status,household_size,o_scorad,FWD_filepath,REV_filepath
#sample-id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9000107,3,F1,SA506,CGTGAGTG,SB701,CTCGACTT,SB701SA506,CTCGACTT-CGTGAGTG,1.010000e+21,F1,20,Yes,Zymo kit,QiaSymphony,21-Mar,Ca-029-TD,case-lesional skin,case,Cape Town,lesional skin,skin,18,male,6/23/2015,Winter,Unexposed,negative,4.0,25,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...
900051,1,E1,SA505,TCATCGAG,SA701,CGAGAGTT,SA701SA505,CGAGAGTT-TCATCGAG,1.010000e+21,E1,20,Yes,Zymo kit,QiaSymphony,21-Mar,Ca-011-LQ,case-lesional skin,case,Cape Town,lesional skin,skin,31,male,4/21/2015,Autumn,Unexposed,negative,7.0,41,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...
900052,1,E2,SA505,TCATCGAG,SA702,GACATAGT,SA702SA505,GACATAGT-TCATCGAG,1.010000e+21,E2,20,Yes,Zymo kit,QiaSymphony,21-Mar,Ca-011-LQ,case-nonlesional skin,case,Cape Town,nonlesional skin,skin,31,male,4/21/2015,Autumn,Unexposed,negative,7.0,41,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...
900053,1,E3,SA505,TCATCGAG,SA703,ACGCTACT,SA703SA505,ACGCTACT-TCATCGAG,1.010000e+21,E3,20,Yes,Zymo kit,QiaSymphony,21-Mar,Ca-011-LQ,case-anterior nares,case,Cape Town,anterior nares,nasal,31,male,4/21/2015,Autumn,Unexposed,negative,7.0,41,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...
900055,3,D12,SA504,CTGCGTGT,SB712,CGTAGCGA,SB712SA504,CGTAGCGA-CTGCGTGT,1.010000e+21,D12,20,Yes,Zymo kit,QiaSymphony,21-Mar,Ca-012-LM,case-nonlesional skin,case,Cape Town,nonlesional skin,skin,15,female,4/22/2015,Spring,Unexposed,negative,4.0,25,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...


# Perform RPCA with Auto Rank Estimation



In [11]:
# perform RPCA with auto rank estimation on the same
np.seterr(divide = 'ignore')
ordination, distance = rpca(biom_tbl)

# extract and view sample ordinations from RPCA result
spca_df = ordination.samples

# Add a case type column into the spca_df using the meta_data and matching by indices by the join function
spca_df = spca_df.join(metadata['case_type'])
spca_df

,PC1,PC2,PC3,case_type
900221,0.019447,0.094025,0.104781,case-lesional skin
900570,0.085857,-0.034721,0.000204,case-nonlesional skin
900092,-0.061123,-0.066987,-0.008467,case-anterior nares
900466,-0.096082,-0.036088,-0.042834,case-anterior nares
9003932,-0.127355,-0.024372,0.094411,control-anterior nares
...,...,...,...,...
900279,0.021523,0.041060,0.007289,control-nonlesional skin
900304,-0.023019,-0.004441,-0.057023,case-anterior nares
900580,-0.016790,-0.032797,0.125734,case-anterior nares
900484,-0.082562,0.030154,-0.029069,case-anterior nares


# Function to generate Convex Hull PCA plots for Beta Diversity

In [15]:
#formats the p_value in the function for generating the plot
def format_p_value(p_value):
    """
    Format a p-value with statistical significance stars.

    Parameters:
        p_value (float): The p-value to format.

    Returns:
        str: A string indicating the significance level with stars and the p-value.
    """
    if not (0 <= p_value <= 1):
        raise ValueError("p-value must be between 0 and 1.")

    if p_value <= 0.001:
        label = '***'
    elif p_value <= 0.01:
        label = '**'
    elif p_value <= 0.05:
        label = '*'
    else:
        label = ''

    return f"{label}p-val = {p_value}"

In [23]:
def generate_plot(spca_df, palette, pnova_res, filename):
    # create beta diversity plot
    mm = 1/25.4
    fig, ax = plt.subplots(1, 1, figsize=(90*mm, 110*mm))

    sns.scatterplot(
        data=spca_df,
        x="PC1",
        y="PC2",
        hue="case_type",
        edgecolor=None,
        palette=palette,
        ax=ax
    )

    for case_type, case_type_df in spca_df.groupby("case_type"):
        color = palette[case_type]

        points = case_type_df[["PC1", "PC2"]].values
        hull = scipy.spatial.ConvexHull(points)

        hull_plot_x = points[hull.vertices, 0]
        hull_plot_y = points[hull.vertices, 1]
        # Connect last point with first point
        hull_plot_x = np.append(hull_plot_x, points[hull.vertices[0], 0])
        hull_plot_y = np.append(hull_plot_y, points[hull.vertices[0], 1])

        ax.plot(
            hull_plot_x,
            hull_plot_y,
            c=color,
            zorder=0
        )
        ax.fill(
            points[hull.vertices, 0],
            points[hull.vertices, 1],
            c=color,
            alpha=0.3
        )


    handles, labels = ax.get_legend_handles_labels()
    plt.legend(
        handles=handles,
        labels=map(str.capitalize, labels),
        frameon=False,
        fontsize = 7
    )

    pc1_pct, pc2_pct, _ = [f"PC{i+1} ({x*100:.2f}%)" for i, x in enumerate(ordination.proportion_explained)]

    ax.set_xlabel(pc1_pct, fontsize = 7)
    ax.set_ylabel(pc2_pct, fontsize = 7)

    yticklabels = [-0.2, -0.1, 0.0, 0.1, 0.2, 0.3]
    yticklocations = yticklabels  # Assuming you want the locations to match the labels
    ax.set_yticks(yticklocations)
    ax.set_yticklabels(yticklabels, fontsize = 7)

    xticklabels = [-0.2, -0.1, 0.0, 0.1, 0.2, 0.3]
    xticklocations = xticklabels  # Assuming you want the locations to match the labels
    ax.set_xticks(xticklocations)
    ax.set_xticklabels(xticklabels, fontsize = 7)

    # calculate permanova F-statistic
    pnova_res = permanova(distance, spca_df, "case_type")
    print(pnova_res)

    ax.text(-0.18, 0.22, 'PERMANOVA', fontsize=7)
    ax.text(-0.18, 0.2, format_p_value(pnova_res["p-value"]), fontsize=7)

    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)

    plt.tight_layout()
    plt.savefig(filename, dpi = 600)
    plt.show()

# Plot All Groups

In [ ]:
# Set the color palette for the groups in the correct order
palette = {
    'control-nonlesional skin': '#3333B3',     # Dark Blue color for Healthy
    'case-nonlesional skin': '#5cbccb',     # Blue color for AD Non-Lesional
    'case-lesional skin': '#f16c52',       # Red color for AD Lesional
    'control-anterior nares': '#008000',   # Green for healthy Nasal
    'case-anterior nares': '#FFC0CB'        # Pink for AD Nasal
}

# calculate permanova F-statistic
pnova_res = permanova(distance, spca_df, "case_type")
print(pnova_res)

#Plot convex hull Beta diversity plot for all groups 
generate_plot(spca_df, palette, pnova_res,  "../plots/Beta_Plots/16S_Beta_Diversity_all_groups.png")

# Plot Skin vs Nares

In [ ]:
# Recategorize the case_type into two buckets skin vs nares
spca_df_sn = spca_df.copy()
spca_df_sn['case_type'] = spca_df_sn['case_type'].apply(lambda x: 'skin' if x.endswith('skin') else 'nares')
# Set the color palette for the groups in the correct order
palette_sn = {
    'skin': '#3333B3',     # Dark Blue
    'nares': '#FFC0CB'        # Pink
}
# 
pnova_res_sn = permanova(distance, spca_df_sn, "case_type")
print(pnova_res_sn)

# Generate the convex hull plot for skin vs nares 
generate_plot(spca_df_sn, palette_sn, pnova_res_sn, "../plots/Beta_Plots/16S_Beta_Diversity_skin_vs_nares.png")


# Plot Control vs Case (skin and nares)

In [ ]:
# Compare Case vs Control Healthy in skin and nares
spca_df_cc = spca_df.copy()
spca_df_cc['case_type'] = spca_df_cc['case_type'].apply(lambda x: 'control' if x.startswith('control') else 'case')
# Set the color palette for the groups in the correct order
palette_cc = {
    'control': '#3333B3',     # Dark Blue
    'case': '#FFC0CB'        # Pink
}

pnova_res_cc = permanova(distance, spca_df_cc, "case_type")
print(pnova_res_cc)

# Generates convex hull of control vs case samples 
generate_plot(spca_df_cc, palette_cc, pnova_res_cc, "../plots/Beta_Plots/16S_Beta_Diversity_control_vs_case.png")


# checks to see if the total of case and control is the same as the total number of samples 254
spca_df_cc.value_counts('case_type', dropna=False)